Modification à faire :
- Programme pour traiter les données json
- Changer le filtrage des catégorie, pas besoin de filtre pandas, ajouter c=id à la fin de l'url
- Expliquer comment définir une bbox correctement
- Compléter la liste des points d'intérêts disponibles
- Retravailler les cellules markdown et les commentaires Python
- Restructurer les paramètres

Importation des librairies utiles

In [1]:
import requests
import json
import pandas as pd
import overpy
import geopy.distance as gd

Création du dictionnaire des Localisation avec leur URL respectives

In [2]:
#Localisation des données
dict_association_localisation = {'AGGLOMÉRATION LE COTENTIN':'https://vigilo.velorution-cherbourg.fr/get_issues.php?',
                                 'AGGLOMÉRATION TROYENNE':'https://vigilo.troyesenselle.fr/get_issues.php?', 
                                 'AIX MARSEILLE PROVENCE METROPOLE':'https://vigilo.velosenville.org/get_issues.php?',
                                 'BASSIN D\'ARCACHON':'https://vigilo.velobassin.org/get_issues.php?',
                                 'BEAUVAISIS':'https://vigilo.vellovaque.fr/get_issues.php?',
                                 'BORDEAUX MÉTROPOLE':'https://vigilo.cycles-manivelles.org/get_issues.php?',
                                 'BREST MÉTROPOLE':'https://vigilo.bapav.org/get_issues.php?',
                                 'EVREUX':'https://vigilo.cube27.org/get_issues.php?',
                                 'GRAND MONTPELLIER':'https://vigilo.jesuisundesdeux.org/get_issues.php?',
                                 'LA ROCHELLE':'https://vigilo.-lr.corolo.fr/get_issues.php?',
                                 'LORIENT AGGLO':'https://vigilo.claav.fr/get_issues.php?',
                                 'MONS':'https://vigilo.geckonline.be/get_issues.php?',
                                 'NANCY AGGLOMÉRATION':'https://vigilo.rigelk.eu/get_issues.php?',
                                 'NANTES MÉTROPOLE':'https://vigilo.placeauvelo-nantes.fr/get_issues.php?',
                                 'NORD-ISÈRE (APIE)':'https://vigilo.apie-asso.net/get_issues.php?',
                                 'PARIS SACLAY':'https://api.vigilo-parissaclay.mdb-idf.org/get_issues.php?',
                                 'PAYS DE CHÂTEAUGIRON':'https://pouliquen.net/vigilo/get_issues.php?',
                                 'PÉRIGUEUX AGGLO':'https://vigilo.velorution-perigourdine.fr/get_issues.php?',
                                 'SAINT MARCELLIN VERCORS ISÈRE COMMUNAUTÉ':'https://vigilo.roulavelo.org/get_issues.php?',
                                 'SAINT-BRIEUC ARMOR AGGLOMÉRATION':'https://api-vigilo.velo-utile.fr/get_issues.php?',
                                 'STRASBOURG':'https://www.strasbourgavelo.eu/get_issues.php?',
                                 'SUD OISE':'https://vigilo.au5v.fr//get_issues.php?',
                                 'THAU':'https://prod-sete.vigilo.velocite-montpellier.fr/get_issues.php?',
                                 'VANNES':'https://vigilo.kaz.bzh/get_issues.php?'
                                }

df_localisation = pd.DataFrame(dict_association_localisation.keys(), columns=["Localisation"])
df_localisation.set_index("Localisation", inplace=True)
df_localisation

""
Localisation
AGGLOMÉRATION LE COTENTIN
AGGLOMÉRATION TROYENNE
AIX MARSEILLE PROVENCE METROPOLE
BASSIN D'ARCACHON
BEAUVAISIS
BORDEAUX MÉTROPOLE
BREST MÉTROPOLE
EVREUX
GRAND MONTPELLIER


Création du dictionnaire avec les différentes catégories de signalements et leur numéro

In [3]:
# URL de l'API pour récupérer la liste des catégories
url_categories = 'https://vigilo-bf7f2.firebaseio.com/categorieslist.json'
resp = requests.get(url_categories)
resp.raise_for_status()

# Convertir la liste de catégories en un dictionnaire avec le nom de la catégorie comme clé et l'identifiant de catégorie comme valeur
categories_dict = {cat["catname"]: str(cat["catid"]) for cat in resp.json()}

# Créer un dataframe à partir du dictionnaire
df_categories = pd.DataFrame(categories_dict.items(), columns=["Catégories", "id"])

# Changer l'index et trier par id
df_categories = df_categories.set_index("id")

# Afficher le dataframe
df_categories


,Catégories
id,
8,Absence d'aménagement
3,Aménagement mal conçu
4,Défaut d'entretien
5,Absence d'arceaux de stationnement
2,Véhicule ou objet gênant
9,"Accident, chute, incident"
6,"Signalisation, marquage"
7,Incivilité récurrente sur la route
10,Vol ou dégradation de vélo


Liste des POI

In [4]:
# Créer un dictionnaire des points d'intérêt et leur identifiant OSM (ex: pour les parkings à vélo : "amenity:bicycle_parking")

poi_dict = {
    "Écoles": "amenity=school",
    "Hôpitaux": "amenity=hospital",
    "Parkings à vélos": "amenity=bicycle_parking",
    "Stations de vélos en libre-service": "amenity=bicycle_rental",
    "Ateliers de réparation de vélos": "shop=bicycle",
    "Passages piétons": "highway=crossing",
    "Feux piétons": "highway=traffic_signals",
    "Places": "highway=pedestrian",
    "Rampes d'accès pour handicapés": "amenity=wheelchair_ramp",
    "Sièges pour handicapés": "amenity=wheelchair_seating",
    "Parkings pour handicapés": "amenity=parking_space",
    "Accès piétonniers": "footway=sidewalk"
}


# Créer un dataframe à partir du dictionnaire
df_poi = pd.DataFrame(poi_dict.items(), columns=["Points d'intérêt", "Identifiant OSM"])

# Changer l'index
df_poi = df_poi.set_index("Identifiant OSM")

# Afficher le dataframe
df_poi

,Points d'intérêt
Identifiant OSM,
amenity=school,Écoles
amenity=hospital,Hôpitaux
amenity=bicycle_parking,Parkings à vélos
amenity=bicycle_rental,Stations de vélos en libre-service
shop=bicycle,Ateliers de réparation de vélos
highway=crossing,Passages piétons
highway=traffic_signals,Feux piétons
highway=pedestrian,Places
amenity=wheelchair_ramp,Rampes d'accès pour handicapés


Initialisation des paramètres

In [5]:
# Paramètres de recherche

# Filtre de recherche par localisation, exemple : localisation = "BREST MÉTROPOLE"
location_filter = "GRAND MONTPELLIER"

# Filtre des catégories à afficher, exemple : category_filter = "Absence d'aménagement" donne la catégorie Absence d'aménagement
category_filter = "Absence d'aménagement"

# Filtre des points d'intérêt à afficher, exemple : poi_filter = "Parking à vélo" donne le point d'intérêt Parking à vélo
poi_name = "Écoles"

# Boîte de recherche pour les points d'intérêt
central_lat = 48.390177
central_lon = -4.496499
delta = 0.03

bbox = [
    central_lat - delta,  # south latitude
    central_lon - delta,  # west longitude
    central_lat + delta,  # north latitude
    central_lon + delta   # east longitude
]

Création du dataframe des données

In [6]:
def data_collection():
    try:
        resp = requests.get(dict_association_localisation[location_filter])
        resp.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        df = pd.DataFrame(resp.json())
        return df[df['categorie'] == categories_dict[category_filter]] if category_filter else df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None
df = data_collection()

Aperçu des données récoltées

In [7]:
df.head(10)

,token,coordinates_lat,coordinates_lon,address,comment,explanation,time,status,group,categorie,approved,cityname
152,C6D5F36E,43.57634925244399,3.928942680358887,", Lattes",manque liaison de 400m entre 2 pistes cyclables,L'avenue Georges Frêche est équipée de 2 belle...,1695219900,0,0,8,1,Lattes
155,2F3C8944,43.614624,3.8519389444444445,"Rue de Las Sorbes, Montpellier",Circulation très dangereuse des vélos en montée,Vélos en montée obligés de zigzaguer entre les...,1695142140,0,0,8,0,Montpellier
215,2E2A429B,43.60409264996277,3.8873768225312233,"de barcelone, Montpellier",manque un miroir routier car croisement dangereux,,1693502280,0,0,8,1,Montpellier
361,A63C6193,43.65878647831918,3.972549363970756,"Rue de la Cadoule, Vendargues",parking à vélo kiné,,1688715369,0,0,8,1,Vendargues
459,77CC0A38,43.53792955525838,3.8646368030458693,"M 185, Villeneuve-lès-Maguelone",traversée dangereuse,,1686986940,0,0,8,1,Villeneuve-lès-Maguelone
639,D2206E9F,43.63113,3.84137,"Avenue Ernest Hemingway, Montpellier",impossible de joindre la pc depuis domaine d'O,,1684148940,0,0,8,1,Montpellier
786,07A43122,43.61419,3.81054,"Rue des Alouettes, Juvignac","fin de pc, débrouillez-vous",,1682325060,0,0,8,1,Juvignac
802,F2466D96,43.6042,3.82164,"Rue de Bionne, Montpellier",voie dangereuse pour les cyclistes et piétons,"pas de piste cyclable, pas de trottoir, rien p...",1682162880,0,0,8,1,Montpellier
862,BA4F08CE,43.614697361193265,3.8297740370035176,"Route de Lodève, Montpellier","pas de pc, cyclistes en danger!!!",,1681745460,0,0,8,1,Montpellier
863,503CB7FA,43.61465779485059,3.8295497372746463,"Route de Lodève, Montpellier",route dangereuse = vélos sur le trottoir,,1681745460,0,0,8,1,Montpellier


In [8]:
# Trier les données en fonction de leur coordonnées GPS, si elles sont dans la boîte de recherche autour du point d'intérêt
def fetch_poi_data(bbox, poi_name, poi_dict):
    # Fetch the tag for the poi_name from the poi_dict
    poi_tag = poi_dict.get(poi_name)
    if poi_tag is None:
        raise ValueError(f"Le point d'intérêt '{poi_name}' n'est pas reconnu.")
    
    # Format the bounding box and tags correctly for the Overpass query
    formatted_bbox = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"
    formatted_poi_tag = poi_tag.replace("=", '="') + '"'

    # Define the Overpass API URL
    url = "http://overpass-api.de/api/interpreter"

    # Construct the query using the formatted bbox and poi_tag
    query = f"""
    [out:json];
    (
        node[{formatted_poi_tag}](bbox:{formatted_bbox});
        way[{formatted_poi_tag}](bbox:{formatted_bbox});
        rel[{formatted_poi_tag}](bbox:{formatted_bbox});
    );
    out center;
    """

    # Send the request to the Overpass API
    response = requests.get(url, params={'data': query})
    response.raise_for_status()  # Raises an HTTPError for bad responses
    return response.json()


pois = fetch_poi_data(bbox, poi_name, poi_dict)
pois


{'version': 0.6,
 'generator': 'Overpass API 0.7.62.1 084b4234',
 'osm3s': {'timestamp_osm_base': '2024-04-12T21:00:13Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 266168377,
   'lat': 40.3125316,
   'lon': -4.5141042,
   'tags': {'addr:postcode': '28649',
    'amenity': 'school',
    'email': 'colegioseminario@rozascs.org',
    'fax': '+34918611356',
    'isced:level': '2',
    'name': 'Centro Privado de Educación Primaria y Secundaria Seminario Arzobispal la Inmaculada y San Dámaso',
    'operator:type': 'private',
    'phone': '+34918610200',
    'ref:educamadrid': '28029445',
    'short_name': 'CPEPS Seminario Arzobispal la Inmaculada y San Dámaso',
    'website': 'https://www.rozascs.org/'}},
  {'type': 'node',
   'id': 266169847,
   'lat': 40.3106294,
   'lon': -4.4927751,
   'tags': {'addr:housenumber': '48',
    'addr:postcode': '28649',
    'addr:street': '